In [69]:
! pip install -Uqq fastbook

In [70]:
import torch
from fastbook import *

In [71]:
path = untar_data(URLs.MNIST_SAMPLE)

In [72]:
Path.BASE_PATH = path

In [73]:
path.ls()

(#3) [Path('valid'),Path('labels.csv'),Path('train')]

In [74]:
(path/'train').ls()

(#2) [Path('train/7'),Path('train/3')]

In [75]:
threes = [tensor(Image.open(o)) for o in (path/'train'/'3').ls().sorted()]
sevens = [tensor(Image.open(o)) for o in (path/'train'/'7').ls().sorted()]

In [76]:
train_x = torch.cat([torch.stack(threes).float()/255, torch.stack(sevens).float()/255]).view(-1, 28*28)
train_x.shape

torch.Size([12396, 784])

In [77]:
train_y = tensor([1.,0.]*len(threes) + [0., 1.]*len(sevens)).view(-1, 2)
train_y.shape

torch.Size([12396, 2])

In [78]:
def binary_cross_entropy(preds, targ):
    preds = preds.sigmoid()
    return -torch.where(targ==1., preds, 1-preds).log().mean()

In [79]:
def init_params(size, std=1.):
    return (torch.randn(size)*std).requires_grad_()

In [80]:
w = init_params((28*28, 2))
b = init_params(2)

In [81]:
w.shape

torch.Size([784, 2])

In [82]:
def linear(xb): return xb@w + b

In [83]:
preds = linear(train_x[:5])
preds, preds.sigmoid()

(tensor([[ -6.9969,   6.8148],
         [-14.1226,  11.8438],
         [ -6.3983,   3.4259],
         [ -7.2866,  -3.3239],
         [ -4.9341,  10.7746]], grad_fn=<AddBackward0>),
 tensor([[9.1388e-04, 9.9890e-01],
         [7.3559e-07, 9.9999e-01],
         [1.6616e-03, 9.6850e-01],
         [6.8416e-04, 3.4762e-02],
         [7.1454e-03, 9.9998e-01]], grad_fn=<SigmoidBackward0>))

In [84]:
train_y[:5]

tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])

In [85]:
xb, yb = train_x[:5], train_y[:5]
acts = linear(xb)

binary_cross_entropy(acts, yb), torch.nn.BCEWithLogitsLoss()(acts, yb)

(tensor(7.2678, grad_fn=<NegBackward0>),
 tensor(7.2677, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>))

In [86]:
def step(xb, yb, model):
    preds = model(xb)
    loss = binary_cross_entropy(preds, yb)
    loss.backward()

In [87]:
step(train_x, train_y, linear)
w.grad, b.grad

(tensor([[0., nan],
         [0., nan],
         [0., nan],
         ...,
         [0., nan],
         [0., nan],
         [0., nan]]),
 tensor([-0.0984,     nan]))

In [37]:
def batch_accuracy(preds, targ):
    preds = preds.sigmoid()
    return ((preds > 0.5) == targ.bool()).float().mean()

In [38]:
batch_accuracy(linear(train_x[:5]), train_y[:5])

tensor(1.)

In [111]:
preds = linear(train_x[:5])
preds

tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<AddBackward0>)